## Código final consolidado: LSTM com HPO + linearidade + robustez

In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Flatten, Concatenate, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1_l2
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from kerastuner.tuners import RandomSearch
import joblib
import os

# === Função para criar dataset multi-step ===
def criar_dataset_multi_step(series, look_back=10, passo=1):
    X, y = [], []
    for i in range(len(series) - look_back - passo + 1):
        X.append(series[i:i+look_back])
        y.append(series[i+look_back+passo-1])
    return np.array(X), np.array(y)

# === Função geradora de modelo com look_back fixado ===
def get_build_model(look_back):
    def build_model(hp):
        inputs = Input(shape=(look_back, 1))
        lstm_out = LSTM(
            units=hp.Int('lstm_units', min_value=32, max_value=128, step=32),
            return_sequences=False
        )(inputs)

        flattened = Flatten()(inputs)
        normalized = BatchNormalization()(flattened)
        linear_weights = Dense(
            1, use_bias=False,
            kernel_regularizer=l1_l2(l1=0.01, l2=0.01),
            name="linear_combination"
        )(normalized)

        combined = Concatenate()([lstm_out, linear_weights])
        output = Dense(1)(combined)

        model = Model(inputs, output)
        model.compile(
            optimizer='adam',
            loss=tf.keras.losses.Huber(delta=1.0),
            metrics=['mae']
        )
        return model
    return build_model

# === Loop para cada horizonte de previsão ===
a3_df = pd.read_csv("A3_component.csv")
a3 = a3_df["A3"].values.reshape(-1, 1)

for passo in [1, 5, 7, 30]:
    look_back = 5 if passo <= 5 else 10
    print(f"\n🚀 Treinando LSTM para t+{passo} com look_back={look_back}")

    # Normalização
    scaler = MinMaxScaler()
    a3_scaled = scaler.fit_transform(a3)
    joblib.dump(scaler, f"scaler_A3_t{passo}.joblib")

    # Criar dataset
    X, y = criar_dataset_multi_step(a3_scaled, look_back=look_back, passo=passo)

    # Tuner
    tuner = RandomSearch(
        get_build_model(look_back),
        objective='val_loss',
        max_trials=5,
        executions_per_trial=1,
        directory='tuner_results',
        project_name=f'lstm_t{passo}'
    )

    # Validação cruzada temporal
    tscv = TimeSeriesSplit(n_splits=5)
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        tuner.search(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=50,
            batch_size=32,
            callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
            verbose=0
        )
        break  # usar apenas a 1ª divisão do TimeSeriesSplit

    # Treinar melhor modelo
    best_model = tuner.get_best_models(num_models=1)[0]
    best_model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=100,
        batch_size=32,
        callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
        verbose=1
    )

    # Salvar modelo
    best_model.save(f"lstm_a3_t{passo}.keras")
    print(f"✅ Modelo salvo: lstm_a3_t{passo}.keras")



🚀 Treinando LSTM para t+1 com look_back=5
Reloading Tuner from tuner_results\lstm_t1\tuner0.json
Epoch 1/100


C:\Users\marco\AppData\Roaming\Python\Python311\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


42/42 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0135 - mae: 0.0633 - val_loss: 0.0105 - val_mae: 0.0507
Epoch 2/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0110 - mae: 0.0562 - val_loss: 0.0092 - val_mae: 0.0520
Epoch 3/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0093 - mae: 0.0526 - val_loss: 0.0077 - val_mae: 0.0464
Epoch 4/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0077 - mae: 0.0467 - val_loss: 0.0064 - val_mae: 0.0437
Epoch 5/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0061 - mae: 0.0403 - val_loss: 0.0052 - val_mae: 0.0412
Epoch 6/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0048 - mae: 0.0367 - val_loss: 0.0045 - val_mae: 0.0480
Epoch 7/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0038 - mae: 0.0369 - val_loss: 0.0030 - val_mae: 0.0287
Epoch 8/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0031 - mae: 0.0326 - val_loss: 0.0024 - val_mae: 0.0262
Epoch 9/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0023 - mae: 0.

C:\Users\marco\AppData\Roaming\Python\Python311\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


42/42 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 0.0182 - mae: 0.0998 - val_loss: 0.0180 - val_mae: 0.1162
Epoch 2/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0159 - mae: 0.0990 - val_loss: 0.0151 - val_mae: 0.1047
Epoch 3/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0149 - mae: 0.0969 - val_loss: 0.0149 - val_mae: 0.1113
Epoch 4/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0133 - mae: 0.0969 - val_loss: 0.0137 - val_mae: 0.1083
Epoch 5/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0129 - mae: 0.0956 - val_loss: 0.0138 - val_mae: 0.1135
Epoch 6/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0116 - mae: 0.0914 - val_loss: 0.0130 - val_mae: 0.1122
Epoch 7/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0110 - mae: 0.0927 - val_loss: 0.0122 - val_mae: 0.1104
Epoch 8/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0103 - mae: 0.0924 - val_loss: 0.0119 - val_mae: 0.1119
Epoch 9/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0101 - mae: 0.

C:\Users\marco\AppData\Roaming\Python\Python311\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0290 - mae: 0.1175 - val_loss: 0.0288 - val_mae: 0.1428
Epoch 2/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0239 - mae: 0.1110 - val_loss: 0.0250 - val_mae: 0.1417
Epoch 3/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0203 - mae: 0.1121 - val_loss: 0.0194 - val_mae: 0.1270
Epoch 4/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0158 - mae: 0.1018 - val_loss: 0.0166 - val_mae: 0.1245
Epoch 5/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0148 - mae: 0.1085 - val_loss: 0.0170 - val_mae: 0.1369
Epoch 6/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0125 - mae: 0.1051 - val_loss: 0.0135 - val_mae: 0.1212
Epoch 7/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0117 - mae: 0.1031 - val_loss: 0.0171 - val_mae: 0.1458
Epoch 8/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0112 - mae: 0.1041 - val_loss: 0.0154 - val_mae: 0.1386
Epoch 9/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0098 - mae: 0.

C:\Users\marco\AppData\Roaming\Python\Python311\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0236 - mae: 0.1092 - val_loss: 0.0244 - val_mae: 0.1304
Epoch 2/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0205 - mae: 0.1047 - val_loss: 0.0271 - val_mae: 0.1563
Epoch 3/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0189 - mae: 0.1048 - val_loss: 0.0244 - val_mae: 0.1507
Epoch 4/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0168 - mae: 0.1013 - val_loss: 0.0217 - val_mae: 0.1441
Epoch 5/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0151 - mae: 0.0993 - val_loss: 0.0204 - val_mae: 0.1448
Epoch 6/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0140 - mae: 0.1039 - val_loss: 0.0191 - val_mae: 0.1449
Epoch 7/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0129 - mae: 0.1048 - val_loss: 0.0161 - val_mae: 0.1353
Epoch 8/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0119 - mae: 0.1044 - val_loss: 0.0182 - val_mae: 0.1511
Epoch 9/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0110 - mae: 0.